# Hash Tables

>**Unidade curricular | Course Unit** Estrutura de Dados e Algoritmos | Data Structures and Algorithms
>
>**Professor** Luis Ramada Pereira
>
>**Curso|Course** LCD-PL -- **Ano letivo|Year** 2019/2020
>
>**Autores|Authors**
>
>     | Catarina Castanheira | nº 92478 |
>     | João Martins         | nº 93259 |
>     | Joel Paula           | nº 93392 |
>
>
>27-05-2020

## Implementing a User Dictionary, using a Hash Table. 

Class interface definition:

| Method | Description |
|--------|-------------|
| get(key) | Returns the value associated to the given key. |
| put(key) = value | Associate value `value` to the given key. If key already exists, replace value with the given value. |
| delete(key) | Removes the value with the given key. Should raise an error if key doesn't exist. |
| collisions() | Returns the numbers of collisions that happened in the table. |

The challenge is to use a hash table with size `N`. Size is given when instanciating the UserDictionary. Hash values should be computed using Python's `Hash()`function and then compressed using N modulus division. Collisions should be handled using separate chaining.

## Implementation

We are storing the elements in a Python list, private to our class instance. The list will be sized according to the given initial value `N`, given when instancing the dictionary class.
The list is initialized with `None` in each position.

When adding a new item to the dictionary, we compute the key's hash and compress it. Since we are compressing it by getting the modulus of the list's size - the remainder of the division of the hash by `N` - it basically gives us a number between `0` and `N-1` (the size of the list we are using to store the elements). That number is the index to a position in our list.

Basically, we should store the key and its value on that position. We are using a `(key, value)` Tuple for that, but we could have a specific entry class, for example. 

You can see already that we will have a lot of collisions. For example, if the initial size is 10, a lot of different keys will be mapped to one of those 10 numbers. How do we handle collisions?

If there's an item already occupying the position we should store the new item in, we must stop using that position as a direct storage, and use it to store a list of items instead - the separate _chaining_. That list/chain will grow as much as need, to hold all the items that have keys with the same compressed hash.

A special case is when you are adding an item with the same exact key as an item that is already in our dictionary. We must substitute the value associated to that key by the new given value. That is why we created the `safe_insert()` static method - to update a value for the same key, if it already exists in our dictionary (or in that particular _chain_), or to add if the key is new.



In [1]:
class UserDict:
    def __init__(self, N, debug=False):
        self.__disp = [None] * N
        self.__N = N
        self.__collisions = 0
        self.__debug = debug
    
    @property
    def collisions(self):
        return self.__collisions

    def __getitem__(self, key):
        index = self.__hash(key)
        if self.__disp[index] is None:
            raise KeyError
        elif type(self.__disp[index]) == tuple:
            return self.__disp[index][1]
        elif type(self.__disp[index]) == list:
            res = next( (t for t in self.__disp[index] if t[0] == key), None)
            if res is None:
                raise KeyError
            else:
                return res[1]



    def __setitem__(self, key, value):
        h = self.__hash(key)
        self.__storevalue(h, key, value)

    def __storevalue(self, index, key, value):
        if self.__disp[index] is None:
            self.__disp[index] = (key, value)
        elif self.__disp[index][0] == key:
            self.__disp[index] = (key, value)
        else:
            self.__collisions +=1
            if type(self.__disp[index]) == list:
                self.safe_insert(self.__disp[index], key, value)
            elif type(self.__disp[index]) == tuple:
                lst = [ self.__disp[index] ]
                self.safe_insert(lst, key, value)
                self.__disp[index] = lst
    
    @staticmethod
    def safe_insert(lst, key, value):
        found = False
        for i in range(len(lst)):
            if lst[i][0] == key:
                lst[i] = (key, value)
                found = True
                break
        if not found:
            lst.append((key, value))

    def __hash(self, key):
        h = hash(key)
        h = h % self.__N
        if self.__debug: print(f"key='{key}'; compressed-hash={h}")
        return h

    def __str__(self):
        return str(self.__disp)

    def __repr__(self):
        return str(self)

# Tests
d = UserDict(10, True)
d["abc"] = "João"
assert d["abc"] == "João"
d["cba"] = "Catarina"
assert d["cba"] == "Catarina"
# Test the updating of an existing key directly on the storage list
d["cba"] = "Célia"
assert d["cba"] == "Célia"
d["cbo"] = "Joel"
assert d["cbo"] == "Joel"
d["cbd"] = "Sérgio"
assert d["cbd"] == "Sérgio"
# Test updating an existing key that is already in a chain
d["abc"] = "Luís"
assert d["abc"] == "Luís"
# exception When the key doesn't exist
try:
    assert d["xpto"] 
except KeyError:
    print("Exception catched! Good!")
# exception When the key doesn't exist in an existing chain
try:
    assert d["abj"] 
except KeyError:
    print("Exception catched! Good!")

print(d)
print("Collisions:", d.collisions)

key='abc'; compressed-hash=2
key='abc'; compressed-hash=2
key='cba'; compressed-hash=7
key='cba'; compressed-hash=7
key='cba'; compressed-hash=7
key='cba'; compressed-hash=7
key='cbo'; compressed-hash=9
key='cbo'; compressed-hash=9
key='cbd'; compressed-hash=0
key='cbd'; compressed-hash=0
key='abc'; compressed-hash=2
key='abc'; compressed-hash=2
key='xpto'; compressed-hash=5
Exception catched! Good!
key='abj'; compressed-hash=5
Exception catched! Good!
[('cbd', 'Sérgio'), None, ('abc', 'Luís'), None, None, None, None, ('cba', 'Célia'), None, ('cbo', 'Joel')]
Collisions: 0


Now, to make a proper test, we are going to load a 10 000 words file and fill the dictionary with it.
We are using the file [PalavrasPortuguesas.txt](PalavrasPortuguesas.txt)
(portuguese words).

In [2]:
cnt = 0
dic = UserDict(10000)
with open("PalavrasPortuguesas.txt") as f:
    for line in f:
        cnt += 1
        line =  str.rstrip(line)
        dic[line]= line
        if cnt % 1000 == 0:
            print("Collisions:", dic.collisions, f"[{cnt}]")
        if cnt >= 10000:
            break


Collisions: 39 [1000]
Collisions: 177 [2000]
Collisions: 394 [3000]
Collisions: 690 [4000]
Collisions: 1055 [5000]
Collisions: 1478 [6000]
Collisions: 1945 [7000]
Collisions: 2443 [8000]
Collisions: 3019 [9000]
Collisions: 3628 [10000]


## New hash compression algorithm

Let's try a new compression algorithm:  ((a*k + b) % p) % N. Where:
 - `N` is the initial size;
 - `p` is the first prime number superior to `N`;
 - `a` and `b` are random numbers in the range `{1, p-1}`;
 - `k` is the key's hash.

For that we need a way to find prime numbers:

In [3]:
def is_multiple(a, b):
    return a % b == 0


def number_of_divisors(n):
    divisor = 1           # is an iterator
    divisor_counter = 0   # is a counter

    while divisor <= n:
        if is_multiple(n, divisor):
            divisor_counter += 1
        divisor += 1

    return divisor_counter

def is_prime(n):
    return number_of_divisors(n) == 2

def get_next_prime(n):
    n += 1
    while not is_prime(n):
        n += 1
    else:
        return n


Now we should implement the new dictionary class:

In [21]:
import random

class UserDict2:
    def __init__(self, N, debug=False):
        self.__disp = [None] * N
        self.__collisions = 0
        self.__debug = debug
        self.__N = N
        # get the hash compression seeds now
        self.__p = get_next_prime(N)
        print("Next prime number", self.__p )
        self.__a = random.randint(1, self.__p -1)
        self.__b = random.randint(1, self.__p -1)

    
    @property
    def collisions(self):
        return self.__collisions

    def __getitem__(self, key):
        index = self.__hash(key)
        if self.__disp[index] is None:
            raise KeyError
        elif type(self.__disp[index]) == tuple:
            return self.__disp[index][1]
        elif type(self.__disp[index]) == list:
            res = next( (t for t in self.__disp[index] if t[0] == key), None)
            if res is None:
                raise KeyError
            else:
                return res[1]



    def __setitem__(self, key, value):
        h = self.__hash(key)
        self.__storevalue(h, key, value)

    def __storevalue(self, index, key, value):
        if self.__disp[index] is None:
            self.__disp[index] = (key, value)
        elif self.__disp[index][0] == key:
            self.__disp[index] = (key, value)
        else:
            self.__collisions +=1
            if type(self.__disp[index]) == list:
                self.safe_insert(self.__disp[index], key, value)
            elif type(self.__disp[index]) == tuple:
                lst = [ self.__disp[index] ]
                self.safe_insert(lst, key, value)
                self.__disp[index] = lst
    
    @staticmethod
    def safe_insert(lst, key, value):
        found = False
        for i in range(len(lst)):
            if lst[i][0] == key:
                lst[i] = (key, value)
                found = True
                break
        if not found:
            lst.append((key, value))

    def __hash(self, key):
        h = hash(key)
        h = ((self.__a * h + self.__b) % self.__p) % self.__N
        if self.__debug: print(f"key='{key}'; compressed-hash={h}")
        return h

    def __str__(self):
        return str(self.__disp)

    def __repr__(self):
        return str(self)

Now, let's run the same experience and look at the collisions:

In [22]:
cnt = 0
dic = UserDict2(10000)
with open("PalavrasPortuguesas.txt") as f:
    for line in f:
        cnt += 1
        line =  str.rstrip(line)
        dic[line]= line
        if cnt % 1000 == 0:
            print("Collisions:", dic.collisions, f"[{cnt}]")
        if cnt >= 10000:
            break

Next prime number 10007
Collisions: 42 [1000]
Collisions: 180 [2000]
Collisions: 414 [3000]
Collisions: 705 [4000]
Collisions: 1081 [5000]
Collisions: 1495 [6000]
Collisions: 1975 [7000]
Collisions: 2489 [8000]
Collisions: 3069 [9000]
Collisions: 3689 [10000]
